# Import Dependencies 

In [5]:
from transformers import BertTokenizer
from transformers import BertModel
from sklearn.model_selection import train_test_split
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from torch.nn.utils.clip_grad import clip_grad_norm
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader

import torch.nn as nn
import numpy as np
import re
import pandas as pd
import torch

# Data Processing

In [7]:
df = pd.read_csv('../data/playlist_features_filtered.csv', delimiter=',', index_col=0)

In [14]:
# tokenize the playlist names 
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

names = df.name.to_list()
print(names[315])

encoded_playlist_names = tokenizer(text = df.name.to_list(),        
                           add_special_tokens=True,
                           padding = 'max_length',
                           truncation = 'longest_first',
                           max_length = 300,
                           return_attention_mask = True)

input_ids = encoded_playlist_names['input_ids']
attention_masks = encoded_playlist_names['attention_mask']

rap 90s (top 100)


ValueError: Input is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.

In [ ]:
# put data into numpy arrays
names = np.array(input_ids)
input_ids = np.array(input_ids)
attention_masks = np.array(attention_masks)
labels = df[
        ["acousticness","danceability","duration_ms","energy","instrumentalness","key","liveness","loudness","mode","speechiness","tempo","time_signature","valence"]
    ].to_numpy()

In [ ]:
#formatting the input
test_size = 0.1
seed = 42
train_inputs, test_inputs, train_labels, test_labels = \
            train_test_split(input_ids, labels, test_size=test_size, 
                             random_state=seed)
train_masks, test_masks, _, _ = train_test_split(attention_masks, 
                                        labels, test_size=test_size, 
                                        random_state=seed)

In [ ]:
#scale the label scores
score_scaler = StandardScaler()
score_scaler.fit(train_labels)

train_labels = score_scaler.transform(train_labels)
test_labels = score_scaler.transform(test_labels)